In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import requests


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


data/accidents-in-france-from-2005-to-2016/users.csv
data/accidents-in-france-from-2005-to-2016/places.csv
data/accidents-in-france-from-2005-to-2016/holidays.csv
data/accidents-in-france-from-2005-to-2016/caracteristics.csv
data/accidents-in-france-from-2005-to-2016/vehicles.csv


In [2]:
caracteristics_path = 'data/accidents-in-france-from-2005-to-2016/caracteristics.csv'
caracteristics = pd.read_csv(caracteristics_path, encoding = 'latin-1')

/opt/miniconda3/envs/scapper/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
def invalide_rows(caracteristics):
    
    
    adr_na = caracteristics['adr'].isna()
    
    lat_na = caracteristics['lat'].isna() | caracteristics['lat'].isnull()
    long_na = caracteristics['long'].isna() | caracteristics['long'].isnull()
    
    
    lat_zero = caracteristics['lat'] == 0
    long_zero = (caracteristics['long'] == 0) | (caracteristics['long'] == '0')
    
    invalid_adr =  adr_na & (lat_zero | long_zero | lat_na | long_na)
    
    return invalid_adr

In [4]:
invalid_rows = invalide_rows(caracteristics)
valid_rows = ~invalid_rows

In [5]:
valide_caracteristics = caracteristics[valid_rows]
# garder tous ce qui sont dans la metropole
in_metropole = (valide_caracteristics['gps'] == 'M') | valide_caracteristics['gps'].isna() & (valide_caracteristics['dep'] < 951)
valide_caracteristics = valide_caracteristics[in_metropole]

In [6]:
def adr_could_be_filled(valide_caracteristics):
    
    adr_not_na = valide_caracteristics['adr'].notna()
    
    lat_na = valide_caracteristics['lat'].isna() | valide_caracteristics['lat'].isnull()
    long_na = valide_caracteristics['long'].isna() | valide_caracteristics['long'].isnull()
    
    code_com_dep_na  = valide_caracteristics['com'].isna() | valide_caracteristics['com'].isnull() 
    code_com_dep_na |= valide_caracteristics['dep'].isna() | valide_caracteristics['dep'].isnull()

    
    lat_zero = valide_caracteristics['lat'] == 0
    long_zero = (valide_caracteristics['long'] == 0) | (valide_caracteristics['long'] == '0')
    
    return adr_not_na & (~code_com_dep_na) & (lat_na | long_na | lat_zero | long_zero)
    

In [7]:
valide_adr = adr_could_be_filled(valide_caracteristics)

valide_adr_caracteristics = valide_caracteristics[valide_adr]

In [8]:
def formate_com(com):
    com = str(int(com))
    extra_zero = '0' * (3 - len(com))
    return extra_zero + com 


In [9]:
com_new = valide_adr_caracteristics['com'].map(lambda x: formate_com(x))
dep_new = valide_adr_caracteristics['dep'].map(lambda x: str(x)[:2])

valide_adr_caracteristics['dep_com'] = dep_new + com_new

<ipython-input-9-85aa8651ef1a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valide_adr_caracteristics['dep_com'] = dep_new + com_new


In [10]:
import time
import json
import requests
import threading
import numpy as np
import concurrent.futures

In [11]:
def get_columns():
    return [
        ('city', ''), ('name', ''), ('score', float(0.0)), 
        ('postcode', -1), ('longitude', float(0.0)), ('latitude', float(0.0))
    ]


def chunker(df, start, size):
    return df.iloc[start:start+size]


def get_lat_long_from_adr(index_carac):    
    i, carac = index_carac
    
    query_url = 'http://api-adresse.data.gouv.fr/search/?q={}'
    adr_dep_com = '{} {}'.format(carac['adr'], carac['dep_com'])
    url = query_url.format(adr_dep_com)
    resp_json = None
    
    #tries = 0
    #while tries < 3:
        
    #    try:
    response = requests.get(url)
    resp_json = response.json()
            #break
        #except requests.exceptions.ConnectionError:
            # in case of max retry
        #    tries += 1
        #    time.sleep(0.1)
        #except json.JSONDecodeError:
        #    break
    
    num_acc = carac['Num_Acc']
    
    if resp_json is None or len(resp_json['features']) < 1: 
        return [num_acc] + list(np.array(get_columns())[:, 1])
    feature = resp_json['features'][0]
    
    city = feature['properties']['city']
    name = feature['properties']['name']
    score = round(feature['properties']['score'], 3)
    postcode = feature['properties']['postcode']
    longitude = feature['geometry']['coordinates'][0]
    latitude = feature['geometry']['coordinates'][1]
    
    return [num_acc, city, postcode, name, score, longitude, latitude]

In [20]:
start_time = time.time()
valide_adr_carac = valide_adr_caracteristics.iloc[:10053]
cols_to_save = ['Num_Acc'] + list(np.array(get_columns())[:, 0])


chunk_size = 500
filename = 'adr_complet.csv'
start = 0


with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:  
    for i in range(start, valide_adr_carac.shape[0], chunk_size):
        chunk = chunker(valide_adr_carac, i, chunk_size)
        results = []
        for res in executor.map(get_lat_long_from_adr, chunk.iterrows()):
            results.append(res)
               
        with open(filename, 'a') as f:
            df = pd.DataFrame(data=results, columns=cols_to_save)
            df.to_csv(f, mode='a', header=f.tell()==0, index=False)

        if i % chunk_size == 0: 
            print('step', i, '/', valide_adr_carac.shape[0], 'time', (time.time() - start_time) / 60, 'min')


print('total time', (time.time() - start) / 60, 'min')

step 0 / 10053 time 0.19317864974339802 min
step 500 / 10053 time 0.3910294016202291 min
step 1000 / 10053 time 0.5888507684071859 min
step 1500 / 10053 time 0.7619440873463948 min
step 2000 / 10053 time 2.2443670352300007 min
step 2500 / 10053 time 2.4793166001637776 min
step 3000 / 10053 time 2.654086005687714 min


KeyboardInterrupt: 

In [ ]:
#unsent_requests = (grequests.get(u, session=requests.Session(), stream=False) for u in urls)
#valide_adr_caracteristics = add_columns(valide_adr_caracteristics)
#responses = grequests.map(unsent_requests, size=10)
#for i, response in enumerate(responses):
#    lat_long_from_adr(response, valide_adr_caracteristics, i)

In [ ]:
    
    # global valide_adr_carac
    # with lock:
    #    valide_adr_carac.loc[i, 'city'] = feature['properties']['city']
    #    valide_adr_carac.loc[i, 'name'] = feature['properties']['name']
    #    valide_adr_carac.loc[i, 'score'] = round(feature['properties']['score'], 3)
    #    valide_adr_carac.loc[i, 'postcode'] = feature['properties']['postcode']
    #    valide_adr_carac.loc[i, 'longitude'] = feature['geometry']['coordinates'][0]
    #    valide_adr_carac.loc[i, 'latitude'] = feature['geometry']['coordinates'][1]
    
    #if i %1000 == 0:
    #    print("step " , i)